# Desafío 08-07-2019
Integrantes: Yerko Carreño, Javier Pilasi, Daniel Flores, Francisco Fernandez, Máximo Oliva, Rocío Ehijo

__Descripción__

En esta sesión trabajaremos con:
- sbp : Presión Sanguínea Sistólica.
- tobacco : Promedio tabaco consumido por día.
- ldl : Lipoproteína de baja densidad.
- adiposity : Adiposidad.
- famhist : Antecedentes familiares de enfermedades cardiácas. (Binaria)
- types : Personalidad tipo A
- obesity : Obesidad.
- alcohol : Consumo actual de alcohol.
- age : edad.
- chd : Enfermedad coronaria. (dummy)

## Preparar el ambiente de trabajo

- Se detallan los pasos a seguir
- tip: Los tips o sugerencias preceden de tip
- Se generan dos notebooks, uno con las soluciones y otro con los ejercicios.

In [5]:
# Importamos las distintas librerías que utlizaremos en el ejercicio
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import statsmodels.api as sm
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

# Importamos el archivo con el cual trabajaremos
df = pd.read_csv('southafricanheart.csv', index_col=0)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 462 entries, 1 to 463
Data columns (total 10 columns):
sbp          462 non-null int64
tobacco      462 non-null float64
ldl          462 non-null float64
adiposity    462 non-null float64
famhist      462 non-null object
typea        462 non-null int64
obesity      462 non-null float64
alcohol      462 non-null float64
age          462 non-null int64
chd          462 non-null int64
dtypes: float64(5), int64(4), object(1)
memory usage: 39.7+ KB


In [6]:
# Obtenemos las principales características de las variables del DF

df.describe()

,sbp,tobacco,ldl,adiposity,typea,obesity,alcohol,age,chd
count,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000,462.000000
mean,138.326840,3.635649,4.740325,25.406732,53.103896,26.044113,17.044394,42.816017,0.346320
std,20.496317,4.593024,2.070909,7.780699,9.817534,4.213680,24.481059,14.608956,0.476313
min,101.000000,0.000000,0.980000,6.740000,13.000000,14.700000,0.000000,15.000000,0.000000
25%,124.000000,0.052500,3.282500,19.775000,47.000000,22.985000,0.510000,31.000000,0.000000
50%,134.000000,2.000000,4.340000,26.115000,53.000000,25.805000,7.510000,45.000000,0.000000
75%,148.000000,5.500000,5.790000,31.227500,60.000000,28.497500,23.892500,55.000000,1.000000
max,218.000000,31.200000,15.330000,42.490000,78.000000,46.580000,147.190000,64.000000,1.000000


## Pregunta N° 2

A continuación se presenta el siguiente modelo a estimar:
$$log \left( \frac{Pr(chd=1)}{1-Prd(chd=1)} \right) = \beta_0 + \beta_1 famhist$$

Para ello ejecute los siguientes pasos:
1. Recodifique famhist a dummy, asignando 1 a la categoría minoritaria.
2. Utilice smf.logit para estimar el modelo.
3. Implemente una función inverse_logit que realize el mapeo de log-odds a probabilidad.
4. Con el modelo estimado, responda lo siguiente:
    - ¿Cuál es la probabilidad de un individuo con antecedentes familiares de tener una enfermedad coronaria
    - ¿Cuál es la probabilidad de un individuo sin antecedentes familiares de tener una enfermedad coronaria?
    - ¿Cuál es la diferencia en la probabilidad entre un individuo con antecedentes y otro sin antecedentes?
    - Replique el modelo con smf.ols y comente las similitudes entre los coeficientes estimados.
        - tip: Utilice $\beta/4$
    - Estime el mismo modelo con LPM

In [7]:
# Recodificamos la variable  famhist a dummy asignando 1 a la categoría minoritaría
# Primero debemos encontrar cual es la categoría minoritaria
categorica = ['famhist']
for cat in categorica:
    print("\n", cat)
    print(df[cat].value_counts())
    print(df[cat].value_counts()/len(df[cat]))

# Luego de saber cual es la variable minoritaria estamos en condiciones de proceder a su binarización
df['Binfamhist'] = np.where(df['famhist'] == 'Present', 1, 0)


 famhist
Absent     270
Present    192
Name: famhist, dtype: int64
Absent     0.584416
Present    0.415584
Name: famhist, dtype: float64


In [8]:
# Procedemos a generar el modelo logístico solicitado
modelo = smf.logit('chd~Binfamhist', df).fit()
modelo.summary()

Optimization terminated successfully.
         Current function value: 0.608111
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      460
Method:                           MLE   Df Model:                            1
Date:                Sun, 14 Jul 2019   Pseudo R-squ.:                 0.05740
Time:                        17:00:33   Log-Likelihood:                -280.95
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 4.937e-09
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.1690      0.143     -8.169      0.000      -1.449      -0.889
Binfamhist     1.1690      0.203      5.751      0.000       0.771       1.567
==============================================================================
"""

In [49]:
# Procedemos a obtener la media de la variable independiente

media_famhist = df['Binfamhist'].mean()
print("La media del historial familiar es", round(media_famhist, 3))

# Procedemos a evualuar el modelo logistico en el punto promedio

modelo_promedio = (modelo.params['Intercept']
                   + (modelo.params['Binfamhist']
                      * media_famhist))

print("El log odds estimado es de", round(modelo_promedio, 3))

La media del historial familiar es 0.416
El log odds estimado es de -0.683


In [11]:
# Una vez que tenemos nuestro Log odds promedio procedemos a convertirlo a una probabilidad


def invlogit(x):
    return 1/(1+np.exp(-x))


print("La Probabilidad promedio de padecer una enfermedad coronaria cuando se posee un historial familiar de 0.41, es de",
      round(invlogit(modelo_promedio), 2))

La Probabilidad promedio de padecer una enfermedad coronaria cuando se posee un historial familiar de 0.41, es de 0.34


In [12]:
# La Probabilidad de padecer una enfermedad coronaria para alguien con antecedentes familiares es de:

prob_famhist_1 = modelo.params['Intercept']+(modelo.params['Binfamhist']*1)

con_antecedentes = invlogit(prob_famhist_1)

print("La Probabilidad promedio de padecer una enfermedad coronaria para alguien con antecedentes familiares es de",
      con_antecedentes.round(2))

La Probabilidad promedio de padecer una enfermedad coronaria para alguien con antecedentes familiares es de 0.5


In [14]:
# La Probabilidad de padecer una enfermedad coronaria para alguien sin antecedentes familiares es de:

prob_famhist_0 = modelo.params['Intercept']+(modelo.params['Binfamhist']*0)

sin_antecedentes = invlogit(prob_famhist_0)

print("La Probabilidad promedio de padecer una enfermedad coronaria para alguien sin antecedentes familiares es de",
      sin_antecedentes.round(2))

La Probabilidad promedio de padecer una enfermedad coronaria para alguien sin antecedentes familiares es de 0.24


In [21]:
# Para calcular las direncias en las probabilidades simplemente realizamos la resta de ambas probabilidades

print("La diferencia en la probabilidad de enfermedad coronoaia entre un individuo con antecentes y otros sin antecedentes es de",
      round(con_antecedentes-sin_antecedentes,3), "es decir la presencia de historial familiar es un patrón de riesgo para las enfermedades coronarias")

La diferencia en la probabilidad de enfermedad coronoaia entre un individuo con antecentes y otros sin antecedentes es de 0.263 es decir la presencia de historial familiar es un patrón de riesgo para las enfermedades coronarias


In [17]:
#Generamos el modelo con OLS

modelo_ols=smf.ols('chd~Binfamhist',df).fit()
modelo_ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    chd   R-squared:                       0.074
Model:                            OLS   Adj. R-squared:                  0.072
Method:                 Least Squares   F-statistic:                     36.86
Date:                Sun, 14 Jul 2019   Prob (F-statistic):           2.66e-09
Time:                        17:25:36   Log-Likelihood:                -294.59
No. Observations:                 462   AIC:                             593.2
Df Residuals:                     460   BIC:                             601.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2370      0.028      8.489      0.000       0.182       0.292
Binfamhist     0.2630      0.043      6.071      0.000       0.178       0.348
==============================================================================
Omnibus:                      768.898   Durbin-Watson:                   1.961
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               58.778
Skew:                           0.579   Prob(JB):                     1.72e-13
Kurtosis:                       1.692   Cond. No.                         2.47
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
# Ahora comparamos las similitudes entre el modelo sms.logit y sms.ols, para eso dividimos B1 logit por 4

round(modelo.params['Binfamhist']/4,3)


0.292

El coeficiente estimado de logit ajustado (4) es de 0.29, mientras que el coeficiente estimado con ols es de 0,26, existiendo así una diferencia de solo 3 puntos porcentuales, de esta manera, observamos según lo esperado una relativa similitud entre ambos coeficientes, no desviándose fuertemente uno del otro, por lo que podemos concluir que en ambos modelos, tanto logit como en ols, la estimación y pronóstico del modelo son similares, pronosticando magnitiudes similares del efecto de los antecedentes familiares sobre la probabilidad de padecer enfermedades coronarías.

## Estimación completa
Implemente un modelo con la siguiente forma:

$$log \left( \frac{Pr(chd=1)}{1-Prd(chd=1)} \right) = \beta_0 + \sum_{j=1}^N\beta_j X$$

- Depure el modelo manteniendo las variables con significancia estadística al 95%.
- Compare los estadísticos de bondad de ajuste entre ambos.
- Reporte de forma sucinta el efecto de las variables en el log-odds de tener una enfermedad coronaria.

In [36]:
# Implementamos un modelo Logistic con todas las variables de la muestra para determinar la significancias de estas.

modelo_total = (smf.logit(
    '''chd ~ 
        Binfamhist 
        + sbp 
        + tobacco 
        + ldl 
        + adiposity 
        + typea 
        + obesity 
        + alcohol 
        + age''', data=df)
.fit())

modelo_total.summary()

Optimization terminated successfully.
         Current function value: 0.510974
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      452
Method:                           MLE   Df Model:                            9
Date:                Sun, 14 Jul 2019   Pseudo R-squ.:                  0.2080
Time:                        22:31:03   Log-Likelihood:                -236.07
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.055e-22
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.1507      1.308     -4.701      0.000      -8.715      -3.587
Binfamhist     0.9254      0.228      4.061      0.000       0.479       1.372
sbp            0.0065      0.006      1.135      0.256      -0.005       0.018
tobacco        0.0794      0.027      2.984      0.003       0.027       0.132
ldl            0.1739      0.060      2.915      0.004       0.057       0.291
adiposity      0.0186      0.029      0.635      0.526      -0.039       0.076
typea          0.0396      0.012      3.214      0.001       0.015       0.064
obesity       -0.0629      0.044     -1.422      0.155      -0.150       0.024
alcohol        0.0001      0.004      0.027      0.978      -0.009       0.009
age            0.0452      0.012      3.728      0.000       0.021       0.069
==============================================================================
"""

De la observación de la tabla es posible identificar que las variables "sbp", "adiposity", "obesity","alcohol", no son significativamente estadísticos al 95%, por esta razón saldrán de nuestro modelo primario.

In [38]:
# Procedemos a ejecutar el modelo ajustado

modelo_ajustado = (smf.logit(
    '''chd ~ 
          Binfamhist  
          + tobacco 
          + ldl  
          + typea  
          + age''', data=df)
    .fit())
modelo_ajustado.summary()

Optimization terminated successfully.
         Current function value: 0.514811
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                    chd   No. Observations:                  462
Model:                          Logit   Df Residuals:                      456
Method:                           MLE   Df Model:                            5
Date:                Sun, 14 Jul 2019   Pseudo R-squ.:                  0.2020
Time:                        22:33:03   Log-Likelihood:                -237.84
converged:                       True   LL-Null:                       -298.05
Covariance Type:            nonrobust   LLR p-value:                 2.554e-24
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.4464      0.921     -7.000      0.000      -8.251      -4.642
Binfamhist     0.9082      0.226      4.023      0.000       0.466       1.351
tobacco        0.0804      0.026      3.106      0.002       0.030       0.131
ldl            0.1620      0.055      2.947      0.003       0.054       0.270
typea          0.0371      0.012      3.051      0.002       0.013       0.061
age            0.0505      0.010      4.944      0.000       0.030       0.070
==============================================================================
"""

De la comparación de ambas tablas observamos que no existen importantes diferencias en los estadísticos de bondad de ajustes reportados                

In [30]:
# Procedemos a obtener la media de las variable independientes del modelo depurado

media_famhist = df['Binfamhist'].mean()
media_tobacco = df['tobacco'].mean()
media_ldl = df['ldl'].mean()
media_typea = df['typea'].mean()
media_age = df['age'].mean()

# Porcedemos a visualizar las distintas medias de las variables independientes

print("La media del historial familiar es", round(media_famhist, 2))
print("La media del consumo de tobacco es", round(media_tobacco, 2))
print("La media del nivel de ldl es", round(media_ldl, 2))
print("La media del type a es", round(media_typea, 2))
print("La media de age es", round(media_age, 2))

# Procedemos a evualuar el modelo logistico en el punto promedio

modelo_prom_ajustado = (
    modelo.params['Intercept']
    + (modelo_ajustado.params['Binfamhist']
       * media_famhist
       + modelo_ajustado.params['tobacco']
       * media_tobacco
       + modelo_ajustado.params['ldl']
       * media_ldl
       + modelo_ajustado.params['typea']
       * media_typea+modelo_ajustado.params['age']
       * media_age)
)

print("\n El log odds estimado es de", round(modelo_promedio, 3))

# Procedemos a convertir los log odds del modelo en probabilidades

variables = ['Binfamhist', 'tobacco', 'ldl', 'typea', 'age']
inverse = []

for i in variables:
    tmp = (
        modelo_ajustado.params['Intercept']
        + (modelo_ajustado.params[i]
           * df[i].mean()
           )
    )
    inverse.append(round(invlogit(tmp), 5))

dic = dict(zip(variables, inverse))

print("\n", dic)

La media del historial familiar es 0.42
La media del consumo de tobacco es 3.64
La media del nivel de ldl es 4.74
La media del type a es 53.1
La media de age es 42.82

 El log odds estimado es de -0.683

 {'Binfamhist': 0.00231, 'tobacco': 0.00212, 'ldl': 0.00341, 'typea': 0.01126, 'age': 0.01357}


De esta manera podemos observar la contribución de cada variable a la porbabilidad de padecer enfermedad coronaria, del análisis es posible concluir que todas presentes coeficientes positivos, es decir, todas a medidas que aumentan producen un efecto positivo sobre la probabilidad de padecer la enfermedad, sienda la variable que contribuye más al padecimiento de la enfermedad la Edad, la cual por cada año adicional del paciente aumenta en un 1,3% la probabilidad de padecer la enfermedad

## Estimación de perfiles
A partir del modelo depurado, genere las estimaciones en log-odds y posteriormente transfórmelas a probabilidades con inverse_logit . Los perfiles a estimar son los siguientes:
- La probabilidad de tener una enfermedad coronaria para un individuo con características similares a la muestra.
- La probabilidad de tener una enfemerdad coronaria para un individuo con altos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.
- La probabilidad de tener una enfemerdad coronaria para un individuo con bajos niveles de lipoproteína de baja densidad, manteniendo todas las demás características constantes.

In [46]:
# Procedemos a evualuar el modelo logistico en el punto promedio

modelo_prom_ajustado = (
    modelo.params['Intercept']
    + (modelo_ajustado.params['Binfamhist']
       * media_famhist+modelo_ajustado.params['tobacco']
       * media_tobacco+modelo_ajustado.params['ldl']
       * media_ldl+modelo_ajustado.params['typea']
       * media_typea+modelo_ajustado.params['age']
       * media_age)
)

print("El log odds estimado es de", round(modelo_promedio, 3))

# Procedemos a evaluar el modelo en el punto promedio (individuo con caracterísitcas similares a la muestra) transformando el logit con nuestra función inverse

print("La probabilidad de padecer enfermedades coronarías para un indivudo con características similares a la muestra (Promedio de todas las variables) es de",
      round(invlogit(modelo_prom_ajustado), 3))

El log odds estimado es de -0.683
La probabilidad de padecer enfermedades coronarías para un indivudo con características similares a la muestra (Promedio de todas las variables) es de 0.988


In [38]:
# Procedemos a encontrar el individuo con mayores niveles de lipoproteina "ldl" del modelo para evualuarlo en el modelo con las demás variables en el punto promedio
df['ldl'].max()

15.33

In [48]:
# Una vez que ya tenemos nuestro máximo lo evaluamos en el modelo probabístico

modelo_prom_ajustado_ldl_alto = (
    modelo.params['Intercept']
    + (modelo_ajustado.params['Binfamhist']
       * media_famhist
       + modelo_ajustado.params['tobacco']
       * media_tobacco
       + modelo_ajustado.params['ldl']
       * 15.33+modelo_ajustado.params['typea']
       * media_typea
       + modelo_ajustado.params['age']
       * media_age)
)

print("El log odds estimado es de", round(modelo_promedio,3))

print("La probabilidad de padecer enfermedades coronarías para un indivudo con altos niveles de lipoproteina, manteniendo constantes todas las demás variables es de",
      round(invlogit(modelo_prom_ajustado_ldl_alto),3))

El log odds estimado es de -0.683
La probabilidad de padecer enfermedades coronarías para un indivudo con altos niveles de lipoproteina, manteniendo constantes todas las demás variables es de 0.998


De esta manera observamos que la probabilidad de padecer enfermedades coronorías con el máximo nivel de lipopoteina aumenta en torno a un punto porcentual.

In [43]:
#Procedemos a encontrar el individuo con menores niveles de lipoproteina "ldl" del modelo para evualuarlo en el modelo con las demás variables en el punto promedio
df['ldl'].min()

0.98

In [50]:
# Una vez que ya tenemos nuestro mínimo lo evaluamos en el modelo probabístico
modelo_prom_ajustado_ldl_bajo = (
    modelo.params['Intercept']
    + (modelo_ajustado.params['Binfamhist']
       * media_famhist
       + modelo_ajustado.params['tobacco']
       * media_tobacco
       + modelo_ajustado.params['ldl']
       * 0.98
       + modelo_ajustado.params['typea']
       * media_typea+modelo_ajustado.params['age']
       * media_age)
)

print("El log odds estimado es de", round(modelo_promedio,3))
print("La probabilidad de padecer enfermedades coronarías para un indivudo con bajos niveles de lipoproteina, manteniendo constantes todas las demás variables es de",
      round(invlogit(modelo_prom_ajustado_ldl_bajo),3))

El log odds estimado es de -0.683
La probabilidad de padecer enfermedades coronarías para un indivudo con bajos niveles de lipoproteina, manteniendo constantes todas las demás variables es de 0.978


De esta manera observamos que la probabilidad de padecer enfermedades coronarías para un individuo con bajos niveles de lipoproteina cae en torno a un punto porcentual con respecto a un indivuo con niveles promedio